In [19]:
#@title Librerias
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

# Parcial III - Métodos Computacionales

### Nombre:

---

## ⚠ Importante

👁 Puede usar unicamente las librerias vistas en clase (estas están en la parte superior) en caso de agregar una nueva, debe justificarla.

💀 No está permitido el uso de IA's en caso de hacerlo su parcial será **anulado**!

❗ Comente su código y funciones, esto hace parte de la nota.

✅ Responda ordenadamente y con claridad.


---

# 1

(**35 puntos**) El **método de Jacobi** es un algoritmo iterativo para resolver sistemas lineales de la forma:

$$
A \mathbf{x} = \mathbf{b},
$$

donde \$A\$ es una matriz cuadrada no singular. A diferencia del método de Gauss-Seidel, Jacobi utiliza exclusivamente los valores de la iteración anterior para actualizar todas las variables simultáneamente, lo que lo hace más sencillo de paralelizar pero, en general, con una convergencia más lenta.

## Algoritmo

Dado un sistema lineal \$A \mathbf{x} = \mathbf{b}\$, el método de Jacobi consiste en:

1. Elegir un vector inicial \$\mathbf{x}^{(0)}\$ (por ejemplo, el vector nulo).

2. Para cada iteración \$k\$ y cada componente \$i = 1, 2, ..., n\$, actualizar:

$$
x_i^{(k)} = \frac{1}{a_{ii}} \left( b_i - \sum_{\substack{j=1 \\ j \neq i}}^{n} a_{ij} x_j^{(k-1)} \right)
$$

3. Repetir hasta que se cumpla un criterio de convergencia, como:

$$
\| \mathbf{x}^{(k)} - \mathbf{x}^{(k-1)} \| < \text{tolerancia}
$$

---

**a)** Escriba una función llamada `jacobi` que resuelva el sistema lineal \$A \mathbf{x} = \mathbf{b}\$ usando el método iterativo de Jacobi. La función debe aceptar como argumentos:

* La matriz \$A\$ y el vector \$\mathbf{b}\$,
* Una tolerancia (por defecto \$1\times 10^{-10}\$),
* Un número máximo de iteraciones.

Debe retornar la solución aproximada \$\mathbf{x}\$ y el número de iteraciones realizadas.

---

**b)** Aplique su función para resolver el siguiente sistema de ecuaciones lineales:

$$
\begin{cases}
10x_1 - x_2 + 2x_3 = 6 \\[2mm]
-2x_1 + 11x_2 - x_4 = 25 \\[2mm]
3x_1 - x_2 + 10x_3 - x_4 = -11 \\[2mm]
2x_2 - x_3 + 8x_4 = 15
\end{cases}
$$

Use como vector inicial \$\mathbf{x}^{(0)} = \[0, 0, 0, 0]^T\$. Imprima la solución aproximada y el número de iteraciones necesarias para adquirir la convergencia.

---

**c)** Compare su solución con `np.linalg.solve`. ¿Qué tan cercana es la solución iterativa a la exacta? Para esto, realice un gráfico del error usando la norma euclidiana (`np.linalg.norm`) en función de la tolerancia y otro del número de iteraciones, empleando un `np.logspace(-18, -1, 18)`.

---



## Punto a

In [93]:
def Jacobi(A, b, tol = 1e-10, max_iter = 100):
  '''
  Esta función resuelve el sistema Ax = b con el método de Jacobi
  Parametros de entrada:
   -A: ndarray -> Matriz nxn no singular
   -b: ndarray -> Vector nx1
   -max_iter: float -> Máximo de iteraciones deseadas
  Parametros de salida:
   -x: ndarray -> Vector solución
   -i: float -> Número de iteraciones realizadas
  '''
  n = len(b)
  x_inicial = np.zeros(n, dtype = float)
  x_nuevo = np.zeros(n, dtype = float)
  for iter in range(max_iter):
    for i in range(n):
      Suma = 0.0
      for j in range(n):
        if j != i:
          Suma += A[i][j] * x_inicial[j]
      x_nuevo[i] = (b[i] - Suma)/A[i][i]
      if np.linalg.norm(x_nuevo - x_inicial) < tol:
        break
      x_inicial[:] = x_nuevo
  return x_nuevo, iter + 1

## Punto b


In [94]:
A = np.array([
    [10, -1, 2, 0],
    [-2, 11, 0, -1],
    [3, -1, 10, -1],
    [0, 2, -1, 8]
    ], dtype = float)
b = np.array([6, 25, -11, 15], dtype = float)
Sol_Jacobi, iter = Jacobi(A, b)
print('La solución del sistema Ax = b usando Jacobi es:')
print(Sol_Jacobi)
print('\nEl número de iteraciones usadas es:')
print(iter)

La solución del sistema Ax = b usando Jacobi es:
[ 1.06736509  2.56693873 -1.05335607  1.10159581]

El número de iteraciones usadas es:
100


##Punto 3

In [91]:
Sol_linalg = la.solve(A, b) #La solución es muy similar

In [98]:
diferencia = np.linalg.norm(Sol_linalg - Sol_Jacobi)

values = np.logspace(-18, -1, 18)
log.plot(Sol_linalg, Sol_Jacobi)


NameError: name 'log' is not defined

# 2

(**50 puntos**) El sistema **masa-resorte con fricción** es un modelo clásico en física que describe el movimiento de una masa sujeta a una fuerza restauradora (resorte) y una fuerza disipativa (fricción o viscosidad). Su dinámica está gobernada por la ecuación diferencial de segundo orden:

$$
m y''(t) + c y'(t) + k y(t) = 0,
$$

donde:

* $m$ es la masa del objeto,
* $c$ es el coeficiente de fricción (amortiguamiento),
* $k$ es la constante del resorte,
* $y(t)$ es la posición de la masa respecto a su equilibrio.

---

**a)** Reformule esta ecuación como un sistema de primer orden adecuado para ser resuelto con `solve_ivp`. Explique mediante una función explicita y realice su documentación.

---

**b)** Considere el siguiente caso:

* Masa: $m = 1$ kg
* Constante del resorte: $k = 4$ N/m
* Coeficiente de fricción: $c = 0.5$ N·s/m
* Condiciones iniciales: $y(0) = 1$, $y'(0) = 0$
* Intervalo de tiempo: $t \in [0, 20]$

Utilice `solve_ivp` para resolver el sistema y grafique $y(t)$ como $y'(t)$. Interprete el comportamiento del sistema.

---

**c)** Simule dos escenarios adicionales:

1. Sin fricción: $c = 0$
2. Con fuerte fricción: $c = 4.5$

Grafique los tres casos \$y(t)\$ en una misma figura y compare los regímenes: **no amortiguado**, **subamortiguado**, y **sobreamortiguado**.

---

**d)** Para cada uno de los tres casos, calcule la **energía mecánica total** del sistema en función del tiempo. La energía total se define como la suma de energía cinética y potencial:

$$
E(t) = \frac{1}{2} m v^2 + \frac{1}{2} k y^2
$$

Grafique $E(t)$ para los tres escenarios simulados. Analice y compare el comportamiento de la energía en cada caso. ¿Qué sucede con la energía a lo largo del tiempo? ¿Cómo se relaciona esto con el valor del coeficiente de fricción $c$?


In [ ]:
# Aquí va su código

# 3

**(15 puntos)** Sea $A$ una matriz cuadrada de tamaño $n \times n$. Dos propiedades fundamentales de los autovalores de $A$ son:

1. **Determinante**:

   $$
   \det(A) = \prod_{i=1}^{n} \lambda_i
   $$

2. **Traza**:

   $$
   \mathrm{tr}(A) = \sum_{i=1}^{n} \lambda_i
   $$

donde $\lambda_1, \lambda_2, \dots, \lambda_n$ son los autovalores de $A$.

Usando un conjunto de matrices aleatorias $A$ de $5\times 5$ (1000 matrices diferentes), verifique numéricamente ambas propiedades. Puede emplear las funciones `np.linalg.det`, `np.trace` y `np.linalg.eigvals`.


In [120]:
def det_autovals(auto_val):
  multiplicador = 1
  multiplicador_nuevo = 0
  for i in autovalues:
    multiplicador = multiplicador_nuevo
    if len(auto_val) == 1:
      det_A = auto_val
    else:
      multiplicador_nuevo *= auto_val[i] * auto_val[i - 1] * multiplicador
  return multiplicador
det_autovalues = det(autovalues)
print(det_autovalues)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
def traza_autovalues(autovalues):
  suma = 0
  for i in range

In [121]:
A = np.array([
    [10, -1, 2, 0],
    [-2, 11, 0, -1],
    [3, -1, 10, -1],
    [0, 2, -1, 8]
    ], dtype = float)
autovalues = np.linalg.eigvals(A)
det_real = np.linalg.det(A)
traza = np.trace(A)

print(autovalues)
print(det_real)
print(traza)


[13.2415061 +0.j         6.95097033+0.j         9.40376178+0.8703286j
  9.40376178-0.8703286j]
8208.999999999995
39.0
